# Analysis of `wandb` Logs and Metrics
This notebook collects different metrics for a specified experiment.

## Setup

In [1]:
import numpy as np
import pprint

import warnings
warnings.filterwarnings("ignore")

import wandb
api = wandb.Api()

GROUP = "experiment-ta8q2oab"

In [2]:
runs = api.runs("uedyiuajxz-personal/qfl-prod")

In [3]:
grouped_runs = [run for run in runs if run.config.get("group", None) == GROUP]
server = None
clients = []

for run in grouped_runs:
    participant = run.config.get("participant", None)
    if participant == "server":
        server = run
    else:
        clients.append(run)
    print(f"Run ID: {run.id}, Name: {run.name}, Type: {participant}")

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=19 sec). Create a new API with an integer timeout larger than 19, e.g., `api = wandb.Api(timeout=29)` to increase the graphql timeout.


Run ID: td20vnd1, Name: frosty-thunder-174, Type: server
Run ID: 4lnorgyh, Name: silvery-firefly-183, Type: client13
Run ID: klstux4n, Name: leafy-shape-183, Type: client18
Run ID: lk1f9hf1, Name: feasible-bush-183, Type: client14
Run ID: nmppygo2, Name: icy-blaze-178, Type: client19
Run ID: hd1zhh5n, Name: dainty-thunder-183, Type: client7
Run ID: opfaql0r, Name: stoic-jazz-180, Type: client12
Run ID: h4i9f2ik, Name: cosmic-breeze-175, Type: client11
Run ID: fsc2hsqk, Name: graceful-waterfall-175, Type: client3
Run ID: ez40yg0l, Name: summer-shape-183, Type: client0
Run ID: ric2bf54, Name: wandering-forest-175, Type: client6
Run ID: eh0k8vf2, Name: olive-firebrand-175, Type: client15
Run ID: ee0wzdpf, Name: true-brook-175, Type: client16
Run ID: dyfkqvxd, Name: comic-pine-194, Type: client4
Run ID: u1eh36mx, Name: fast-water-175, Type: client5
Run ID: unqtmdg0, Name: warm-firebrand-180, Type: client2
Run ID: 9l0kcv9p, Name: fearless-hill-183, Type: client1
Run ID: 91bd9q8w, Name: peac

## Settings

In [4]:
pp = pprint.PrettyPrinter(indent=1, width=40)
pp.pprint(server.config)

{'batch_size': 32,
 'dataset': 'MRI',
 'fhe_enabled': True,
 'group': 'experiment-ta8q2oab',
 'learning_rate': '1e-3',
 'model': 'fednn',
 'number_clients': 20,
 'participant': 'server',
 'rounds': 20}


## Server and Aggregated Metrics

In [5]:
def compute_statistics(arr: np.ndarray) -> dict:
    """
    Compute basic statistical measures of an array, including mean, median, standard deviation, 
    25th percentile (Q1), and 75th percentile (Q3).

    Parameters
    ----------
    arr : np.ndarray
        Input numerical array.

    Returns
    -------
    dict
        A dictionary containing the following statistics:
        - "mean" : float
            The average of the array.
        - "median" : float
            The middle value of the array.
        - "std" : float
            The standard deviation (sample standard deviation with `ddof=1`).
        - "25%" : float
            The 25th percentile (Q1).
        - "75%" : float
            The 75th percentile (Q3).

    Examples
    --------
    >>> import numpy as np
    >>> data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    >>> compute_statistics(data)
    {'mean': 55.0, 'median': 55.0, 'std': 30.276503540974915, '25%': 32.5, '75%': 77.5}
    """
    if not isinstance(arr, np.ndarray):
        arr = np.array(arr)

    stats = {
        "mean": np.mean(arr),
        "median": np.median(arr),
        "std": np.std(arr, ddof=1),
        "25%": np.percentile(arr, 25),
        "75%": np.percentile(arr, 75),
        "min": np.min(arr),
        "max": np.max(arr),
        "last_value": arr[-1]
    }
    return stats

In [6]:
from rich.console import Console
from rich.table import Table

console = Console()

# Create Table 1 (Basic Stats: Mean, Median, Std)
table1 = Table(title="Server Statistics (Part 1)")
table1.add_column("Metric", justify="left", style="cyan", min_width=20)
table1.add_column("Mean", justify="right", style="green", min_width=12)
table1.add_column("Median", justify="right", style="green", min_width=12)
table1.add_column("Std", justify="right", style="green", min_width=12)

# Create Table 2 (Percentiles + Min/Max/Last Value)
table2 = Table(title="Server Statistics (Part 2)")
table2.add_column("Metric", justify="left", style="cyan", min_width=20)
table2.add_column("25%", justify="right", style="green", min_width=12)
table2.add_column("75%", justify="right", style="green", min_width=12)
table2.add_column("Max", justify="right", style="green", min_width=12)
table2.add_column("Min", justify="right", style="green", min_width=12)
table2.add_column("Last", justify="right", style="green", min_width=12)

server_history = server.history()
for metric in server_history.keys():
    metric_values = [val for val in server_history[metric] if not np.isnan(val)]
    stats = compute_statistics(metric_values)

    # Add row to Table 1
    table1.add_row(
        metric,
        f"{stats['mean']:.2f}",
        f"{stats['median']:.2f}",
        f"{stats['std']:.2f}"
    )

    # Add row to Table 2
    table2.add_row(
        metric,
        f"{stats['25%']:.2f}",
        f"{stats['75%']:.2f}",
        f"{stats['max']:.2f}",
        f"{stats['min']:.2f}",
        f"{stats['last_value']:.2f}"
    )


console.print(table1)
console.print(table2)

                           Server Statistics (Part 1)                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric                     ┃           Mean ┃         Median ┃           Std ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ _step                      │          50.50 │          50.50 │         29.59 │
│ _runtime                   │       63912.08 │       62063.94 │      36778.20 │
│ trainable_parameters       │    12850788.00 │    12850788.00 │           nan │
│ _timestamp                 │  1738217136.51 │  1738215288.38 │      36778.20 │
│ Total Bytes Sent           │   539745433.50 │   539745433.50 │  304112099.73 │
│ Bytes Sent (Round)         │    51404327.00 │    51404327.00 │          0.00 │
│ round                      │          10.50 │          10.50 │          5.92 │
│ round_time                 │        6151.33 │        6151.73 │         84.13 │
│ Bytes Received (Round)     │  1028086540.00 │  1028086540.00 │          0.00 │
│ Total Bytes Received       │ 10794908670.00 │ 10794908670.00 │ 6082241994.57 │
│ parameter_aggregation_time │           0.62 │           0.62 │          0.01 │
│ loss_agg                   │           0.74 │           0.68 │          0.14 │
│ metrics_agg.accuracy       │          86.73 │          88.32 │          4.46 │
│ metrics_agg.precisions     │         100.00 │         100.00 │          0.00 │
│ metrics_agg.f1s            │         100.00 │         100.00 │          0.00 │
│ metrics_agg.recalls        │         100.00 │         100.00 │          0.00 │
│ total_training_time        │      124134.89 │      124134.89 │           nan │
└────────────────────────────┴────────────────┴────────────────┴───────────────┘

                                           Server Statistics (Part 2)                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                     ┃           25% ┃            75% ┃            Max ┃           Min ┃           Last ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ _step                      │         25.25 │          75.75 │         101.00 │          0.00 │         101.00 │
│ _runtime                   │      30953.16 │       93153.86 │      124214.21 │         71.08 │      124214.21 │
│ trainable_parameters       │   12850788.00 │    12850788.00 │    12850788.00 │   12850788.00 │    12850788.00 │
│ _timestamp                 │ 1738184177.57 │  1738246378.30 │  1738277438.65 │ 1738153295.52 │  1738277438.65 │
│ Total Bytes Sent           │  295574880.25 │   783915986.75 │  1028086540.00 │   51404327.00 │  1028086540.00 │
│ Bytes Sent (Round)         │   51404327.00 │    51404327.00 │    51404327.00 │   51404327.00 │    51404327.00 │
│ round                      │          5.75 │          15.25 │          20.00 │          1.00 │          20.00 │
│ round_time                 │       6101.70 │        6174.96 │        6384.66 │       5990.96 │        6148.15 │
│ Bytes Received (Round)     │ 1028086540.00 │  1028086540.00 │  1028086540.00 │ 1028086540.00 │  1028086540.00 │
│ Total Bytes Received       │ 5911497605.00 │ 15678319735.00 │ 20561730800.00 │ 1028086540.00 │ 20561730800.00 │
│ parameter_aggregation_time │          0.61 │           0.62 │           0.66 │          0.59 │           0.60 │
│ loss_agg                   │          0.66 │           0.83 │           0.98 │          0.54 │           0.92 │
│ metrics_agg.accuracy       │         85.92 │          89.48 │          90.91 │         71.64 │          90.91 │
│ metrics_agg.precisions     │        100.00 │         100.00 │         100.00 │        100.00 │         100.00 │
│ metrics_agg.f1s            │        100.00 │         100.00 │         100.00 │        100.00 │         100.00 │
│ metrics_agg.recalls        │        100.00 │         100.00 │         100.00 │        100.00 │         100.00 │
│ total_training_time        │     124134.89 │      124134.89 │      124134.89 │     124134.89 │      124134.89 │
└────────────────────────────┴───────────────┴────────────────┴────────────────┴───────────────┴────────────────┘

## Client Metrics

In [7]:
def compute_client_statistics(client_histories):
    """
    Computes statistics for each metric by merging values across all clients.

    Parameters
    ----------
    client_histories : list[dict]
        A list of dictionaries where each dictionary contains metric history for a single client.

    Returns
    -------
    dict
        A dictionary where each key is a metric name and the value is its computed statistics.
    """
    merged_metrics = {}

    # Iterate through all client histories
    for client_history in client_histories:
        for metric, values in client_history.items():
            # Remove NaN values and flatten lists
            cleaned_values = [val for val in values if not np.isnan(val)]

            # Merge values from all clients
            if metric not in merged_metrics:
                merged_metrics[metric] = []
            merged_metrics[metric].extend(cleaned_values)

    # Compute statistics for each metric
    stats_dict = {metric: compute_statistics(np.array(values)) for metric, values in merged_metrics.items()}

    return stats_dict

# Example: Fetch history from multiple clients
client_histories = [client.history() for client in clients]  # Replace with actual client list

# Compute merged statistics
merged_stats = compute_client_statistics(client_histories)

# Display results using rich
console = Console()

# Create Table 1 (Basic Stats: Mean, Median, Std)
table1 = Table(title="Merged Client Metrics Statistics (Part 1)")
table1.add_column("Metric", justify="left", style="cyan", min_width=20)
table1.add_column("Mean", justify="right", style="green", min_width=12)
table1.add_column("Median", justify="right", style="green", min_width=12)
table1.add_column("Std", justify="right", style="green", min_width=12)

# Create Table 2 (Percentiles + Min/Max/Last Value)
table2 = Table(title="Merged Client Metrics Statistics (Part 2)")
table2.add_column("Metric", justify="left", style="cyan", min_width=20)
table2.add_column("25%", justify="right", style="green", min_width=12)
table2.add_column("75%", justify="right", style="green", min_width=12)
table2.add_column("Max", justify="right", style="green", min_width=12)
table2.add_column("Min", justify="right", style="green", min_width=12)
table2.add_column("Last", justify="right", style="green", min_width=12)

# Populate table with computed statistics
for metric, stats in merged_stats.items():
    table1.add_row(
        metric,
        f"{stats['mean']:.2f}",
        f"{stats['median']:.2f}",
        f"{stats['std']:.2f}"
    )

    # Add row to Table 2
    table2.add_row(
        metric,
        f"{stats['25%']:.2f}",
        f"{stats['75%']:.2f}",
        f"{stats['max']:.2f}",
        f"{stats['min']:.2f}",
        f"{stats['last_value']:.2f}"
    )

# Display table
console.print(table1)
console.print(table2)


               Merged Client Metrics Statistics (Part 1)               
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric               ┃          Mean ┃        Median ┃          Std ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ _timestamp           │ 1738216806.65 │ 1738215266.86 │     35928.39 │
│ decryption_time      │          0.98 │          0.97 │         0.06 │
│ encryption_time      │          0.00 │          0.00 │         0.00 │
│ client_round_time    │       6049.54 │       6040.53 │        99.71 │
│ _step                │         39.50 │         39.50 │        23.10 │
│ _runtime             │      63470.10 │      61930.31 │     35928.39 │
└──────────────────────┴───────────────┴───────────────┴──────────────┘

                               Merged Client Metrics Statistics (Part 2)                                
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric               ┃           25% ┃           75% ┃           Max ┃           Min ┃          Last ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ _timestamp           │ 1738184166.12 │ 1738246345.22 │ 1738277391.43 │ 1738153402.06 │ 1738277390.88 │
│ decryption_time      │          0.95 │          1.00 │          1.50 │          0.91 │          0.95 │
│ encryption_time      │          0.00 │          0.00 │          0.02 │          0.00 │          0.00 │
│ client_round_time    │       5983.24 │       6109.36 │       6380.94 │       5792.86 │       6076.31 │
│ _step                │         19.75 │         59.25 │         79.00 │          0.00 │         79.00 │
│ _runtime             │      30829.57 │      93008.68 │     124054.88 │         65.51 │     124054.33 │
└──────────────────────┴───────────────┴───────────────┴───────────────┴───────────────┴───────────────┘

## Other Computations

In [8]:
some_client = clients[0]
history = some_client.history()
metric_values = [val for val in history["decryption_time"] if not np.isnan(val)]
sum(metric_values)

39.60692310333252